In [1]:
import pandas as pd
import numpy as np
import time
import warnings
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import joblib

# [Configuration] 경고 메시지 출력 방지 (Clean Output)
warnings.filterwarnings('ignore')

print("--- [Final Hybrid Pipeline] Data-Driven Context + Physics-Guided Logic ---")
start_total = time.time()

# ==============================================================================
# [Step 0] 환경 설정 및 하이퍼파라미터
# ==============================================================================
CSV_TRAIN = 'train.csv'
CSV_TEST = 'test.csv'
SAMPLE_SUB = 'submission_sample.csv'
OUTPUT_FILE = 'submission_Final_Hybrid.csv'

# [Hyperparameters] 실험 기반 휴리스틱 튜닝 (Heuristic Tuning)
# 풍부한 시차(Lag) 변수 학습을 위해 모델 복잡도(num_leaves)를 유지하되,
# 과적합 방지를 위해 subsample(행 샘플링)과 colsample(열 샘플링) 적용
LGB_PARAMS = {
    'objective': 'mae',
    'metric': 'mae',
    'n_estimators': 3000,
    'learning_rate': 0.03,
    'num_leaves': 127,
    'subsample': 0.8,
    'colsample_bytree': 0.7,
    'min_data_in_leaf': 50,
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1
}

# ==============================================================================
# [Step 1] 데이터 로드 및 통합 (One-Universe)
# ==============================================================================
print("\n[Step 1] 데이터 로드 및 무결성 확보...")

# 날짜 파싱을 포함한 고속 로드
df_train = pd.read_csv(CSV_TRAIN, parse_dates=['time'])
df_test = pd.read_csv(CSV_TEST, parse_dates=['time'])

# [Safety] 제출 시 순서 섞임 방지를 위한 원본 인덱스 백업
df_test['original_index'] = df_test.index

# 타겟(Target) 분리
y_train = df_train['nins'].copy()

# Data Leakage 방지를 위해 타겟 및 파생 컬럼 제거
drop_cols = ['nins', 'energy']
df_train.drop(columns=[c for c in drop_cols if c in df_train.columns], inplace=True)
df_test.drop(columns=[c for c in drop_cols if c in df_test.columns], inplace=True)

# 데이터셋 구분 태그
df_train['dataset_type'] = 'train'
df_test['dataset_type'] = 'test'

# [Strategy] One-Universe 통합 파이프라인
# Train/Test의 스케일링 및 군집화 기준 통일을 위해 병합 수행
df_full = pd.concat([df_train, df_test], ignore_index=True)
print(f" -> 통합 완료. 총 행 수: {len(df_full)}")


# ==============================================================================
# [Step 2] 결측치 처리 (하이브리드 보간)
# ==============================================================================
print("\n[Step 2] 하이브리드 보간(Hybrid Interpolation) 수행...")

cols_weather = [
    'appr_temp', 'ceiling', 'cloud_b', 'dew_point', 'precip_1h', 'pressure', 
    'real_feel_temp', 'real_feel_temp_shade', 'rel_hum', 'temp_b', 'uv_idx', 'vis', 
    'wind_chill_temp', 'wind_dir_b', 'wind_gust_spd', 'wind_spd_b', 'cloud_a', 
    'ground_press', 'humidity', 'rain', 'snow', 'temp_a', 'temp_max', 'temp_min', 
    'wind_dir_a', 'wind_spd_a'
]
# 실제 존재하는 컬럼만 필터링
cols_exist = [c for c in cols_weather if c in df_full.columns]

# [Logic] 물리적 연속성(Continuity) 보존을 위해 발전소별 시간순 정렬
df_full.sort_values(by=['pv_id', 'time'], inplace=True)

# [Imputation] 시계열 추세를 끊지 않기 위해 Forward/Backward Fill 적용
df_full[cols_exist] = df_full.groupby('pv_id')[cols_exist].ffill().bfill()


# ==============================================================================
# [Step 3] 이중 군집화 (Dual Clustering)
# ==============================================================================
print("\n[Step 3] 이중 군집화 (Dual Clustering) 전략 적용...")
scaler = StandardScaler()

# 3-1. 위치 군집 (Location Cluster) - Micro Segmentation
# 지형적 특성(음영, 고도 등)을 반영하기 위해 K=20으로 세분화
loc_features = ['coord1', 'coord2']
scaled_loc = scaler.fit_transform(df_full[loc_features])
kmeans_loc = KMeans(n_clusters=20, random_state=42, n_init=10)
df_full['cluster_loc'] = kmeans_loc.fit_predict(scaled_loc)
df_full['cluster_loc'] = df_full['cluster_loc'].astype('category')

# 3-2. 기후 군집 (Climate Cluster) - Macro Segmentation
# 지역 전반의 거시적 기상 패턴을 반영하기 위해 K=10 설정
clim_features = ['temp_a', 'humidity', 'wind_spd_a', 'pressure', 'vis']
target_climate = [c for c in clim_features if c in df_full.columns]
pv_stats = df_full.groupby('pv_id')[target_climate].mean() # 발전소별 기후 프로필

scaled_cli = scaler.fit_transform(pv_stats)
kmeans_cli = KMeans(n_clusters=10, random_state=42, n_init=10)
pv_labels = kmeans_cli.fit_predict(scaled_cli)

# 매핑 (Mapping)
climate_map = dict(zip(pv_stats.index, pv_labels))
df_full['cluster_climate'] = df_full['pv_id'].map(climate_map).astype('category')


# ==============================================================================
# [Step 4] 특성 공학 (물리 공식 + Rich Context)
# ==============================================================================
print("\n[Step 4] 하이브리드 특성 공학 (Physics + Data)...")

# 4-1. 시간 변수 (Cyclical Encoding)
df_full['hour'] = df_full['time'].dt.hour
df_full['minute'] = df_full['time'].dt.minute
df_full['day_of_year'] = df_full['time'].dt.dayofyear

# 시간의 연속성을 위해 Sin/Cos 변환
df_full['hour_sin'] = np.sin(2 * np.pi * df_full['hour'] / 24.0)
df_full['hour_cos'] = np.cos(2 * np.pi * df_full['hour'] / 24.0)

# 4-2. [Physics] 태양 기하학(Solar Geometry) 시뮬레이션
# 날짜와 시간에 따른 이론적 태양 에너지 가용량(Daylight Cosine) 계산
# 가정: 한국 시간(KST) 기준 남중 시각(Solar Noon) 약 12:00
solar_noon = 12.0 
day_duration = 12.0 + 2.5 * np.sin(2 * np.pi * (df_full['day_of_year'] - 80) / 365.25)

df_full['decimal_hour'] = df_full['hour'] + df_full['minute'] / 60.0
scaled_time = (df_full['decimal_hour'] - solar_noon) * np.pi / (day_duration / 2)

# 'daylight_cosine'은 Step 7의 야간 마스킹(Night Masking)에 활용됨
df_full['daylight_cosine'] = np.cos(scaled_time) 

# 4-3. [Data-Driven] 대규모 시차(Lag) 변수 생성
# 모델에게 풍부한 문맥(Context)을 제공하기 위해 26개 모든 센서 변수의 과거 데이터 생성
print(" -> 모델 문맥 강화를 위한 Lag 변수 대량 생성 중...")

for col in cols_exist:
    # Lag 6 (30분 전), Lag 12 (1시간 전)
    df_full[f'{col}_lag30m'] = df_full.groupby('pv_id')[col].shift(6)
    df_full[f'{col}_lag1h'] = df_full.groupby('pv_id')[col].shift(12)

print(f" -> 특성 확장 완료. 총 변수 개수: {df_full.shape[1]}")

# Lag 생성으로 인한 초기 결측치는 Backfill로 처리
df_full = df_full.groupby('pv_id').bfill()


# ==============================================================================
# [Step 5] 데이터 분리 (Split)
# ==============================================================================
print("\n[Step 5] 학습/테스트 데이터 분리...")
mask_train = df_full['dataset_type'] == 'train'
mask_test = df_full['dataset_type'] == 'test'

X_train = df_full.loc[mask_train].copy()
X_test = df_full.loc[mask_test].copy()

# 학습 피처 선정 (수치형 + 범주형)
feats_num = list(X_train.select_dtypes(include=['number']).columns)
feats_cat = list(X_train.select_dtypes(include=['category']).columns)
features = feats_num + feats_cat

# 학습 방해 컬럼(메타 데이터) 제외
exclude_cols = {'nins', 'energy', 'pv_id', 'dataset_type', 'time', 'original_index'}
features = [f for f in features if f not in exclude_cols]
cat_feats = [f for f in feats_cat if f in features]

print(f" -> 최종 학습 변수 개수: {len(features)}")


# ==============================================================================
# [Step 6] 모델 학습 (LightGBM)
# ==============================================================================
print("\n[Step 6] LightGBM 모델 학습 시작...")
model = lgb.LGBMRegressor(**LGB_PARAMS)

model.fit(
    X_train[features], y_train,
    eval_set=[(X_train[features], y_train)],
    categorical_feature=cat_feats,
    callbacks=[
        lgb.early_stopping(stopping_rounds=100, verbose=False),
        lgb.log_evaluation(period=1000)
    ]
)


# ==============================================================================
# [Step 7] 예측 및 물리적 후처리 (Safety Mechanisms)
# ==============================================================================
print("\n[Step 7] 추론 및 안전장치 적용...")
preds = model.predict(X_test[features])

# 7-1. 기본 보정 (음수 발전량 제거)
preds = np.maximum(preds, 0)

# 7-2. [Physics] 야간 마스킹 (Daylight Masking)
# 'daylight_cosine' 값이 0 이하(해짐)인 경우 강제로 0 처리하여 물리적 정합성 확보
if 'daylight_cosine' in X_test.columns:
    # 부동소수점 오차 고려한 임계값 설정
    mask_night = X_test['daylight_cosine'] <= 0.0001
    preds[mask_night] = 0
    print(f" -> [후처리] 물리적 밤 시간대 {sum(mask_night)}건을 0으로 보정했습니다.")

# 7-3. [Safety] 인덱스 복원 (Index Restoration)
# 제출 파일의 순서가 원본 test.csv와 완벽하게 일치하도록 재정렬
results = pd.DataFrame({
    'nins': preds,
    'original_index': X_test['original_index'].values
})

results.sort_values(by='original_index', inplace=True)

# ==============================================================================
# [Step 8] 제출 파일 생성
# ==============================================================================
sub = pd.read_csv(SAMPLE_SUB)
sub['nins'] = results['nins'].values
sub.to_csv(OUTPUT_FILE, index=False)

print(f"\n[성공] 최종 제출 파일이 '{OUTPUT_FILE}'로 저장되었습니다.")
print(f"총 소요 시간: {time.time() - start_total:.2f} 초")

--- [Final Hybrid Pipeline] Data-Driven Context + Physics-Guided Logic ---

[Step 1] 데이터 로드 및 무결성 확보...


FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'